In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm
from os.path import join, isfile
from os import listdir
from functools import partial

import glam
import parameter_recovery as pr

%load_ext autoreload
%autoreload 2

/Users/felixmolter/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Systematic parameter recovery – Hierarchical model, with Metropolis

In [2]:
output_folder = join('results', 'parameter-recovery_hierarchical', 'metropolis')
label = 'hierarchical_metropolis'

## Define parameter ranges from 2011 estimates

In [3]:
bounds = dict(v=[0.000015, 0.00015],
              gamma=[-1, 1],
              s=[0.004, 0.011],
              tau=[0.1, 1.25])

In [4]:
# Determine low, medium, high mean values as 10th, 50th and 90th percentile of these ranges
means = {parameter: np.percentile(np.linspace(*bounds[parameter], 100), [10, 50, 90])
          for parameter in ['v', 'gamma', 's', 'tau']}
means

{'v': array([2.850e-05, 8.250e-05, 1.365e-04]),
 'gamma': array([-8.00000000e-01,  5.55111512e-17,  8.00000000e-01]),
 's': array([0.0047, 0.0075, 0.0103]),
 'tau': array([0.215, 0.675, 1.135])}

In [5]:
sds = dict(v=0.00005,
           s=0.001,
           gamma=0.3,
           tau=0.2)

In [6]:
parameter_info = {parameter: dict(mu=dict(low=means[parameter][0],
                                          medium=means[parameter][1],
                                          high=means[parameter][2]),
                                  sd=sds[parameter],
                                  bounds=tuple(bounds[parameter]))
                  for parameter in ['v', 'gamma', 's', 'tau']}

## Run the recovery in parallel

In [7]:
partial_output_folder = join(output_folder, 'partial')

simulate_group_args = dict(kind='hierarchical',
                           n_individuals=20,
                           n_trials=50, 
                           n_items=3,
                           error_weight=0,
                           value_range=[1, 10],
                           seed=None)

make_model_args = dict(kind='hierarchical',
                       t0_val=0,
                       error_weight=0)

fit_args = dict(method='MCMC',
                step=pm.Metropolis,
                draws=5000,
                tune=40000,
                cores=1,
                chains=2,
                progressbar=False)

wrapper = partial(pr.recover_and_save_hierarchical,
                  output_folder=partial_output_folder,
                  label=label,
                  simulate_group_args=simulate_group_args,
                  make_model_args=make_model_args,
                  fit_args=fit_args)

In [8]:
from multiprocessing import Pool

n_cores = 4
p = Pool(n_cores)

output = p.map(wrapper, pr.generate_hierarchical_parameter_sets(parameter_info))

Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...


INFO (theano.gof.compilelock): Waiting for existing lock by process '28522' (I am process '28578')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '28575' (I am process '28577')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '28575' (I am process '28576')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '28524' (I am process '28578')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x

Fitting 1 model(s) using MCMC...
  Fitting model 1 of 1...


INFO (theano.gof.compilelock): Waiting for existing lock by process '28576' (I am process '28578')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '28576' (I am process '28575')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '28524' (I am process '28577')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '28522' (I am process '28576')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x

Fitting 1 model(s) using MCMC...
  Fitting model 1 of 1...


...

/!\ Automatically setting parameter precision...
Generating hierarchical model for 20 subjects...
Fitting 1 model(s) using MCMC...
  Fitting model 1 of 1...


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [tau]
>Metropolis: [tau_sd]
>Metropolis: [tau_mu]
>Metropolis: [s]
>Metropolis: [s_sd]
>Metropolis: [s_mu]
>Metropolis: [gamma]
>Metropolis: [gamma_sd]
>Metropolis: [gamma_mu]
>Metropolis: [v]
>Metropolis: [v_sd]
>Metropolis: [v_mu]
The gelman-rubin statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


/!\ Automatically setting parameter precision...
Generating hierarchical model for 20 subjects...
Fitting 1 model(s) using MCMC...
  Fitting model 1 of 1...


Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [tau]
>Metropolis: [tau_sd]
>Metropolis: [tau_mu]
>Metropolis: [s]
>Metropolis: [s_sd]
>Metropolis: [s_mu]
>Metropolis: [gamma]
>Metropolis: [gamma_sd]
>Metropolis: [gamma_mu]
>Metropolis: [v]
>Metropolis: [v_sd]
>Metropolis: [v_mu]
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


/!\ Automatically setting parameter precision...


## Combine results

In [10]:
filenames = [file for file in listdir(partial_output_folder)
             if file.endswith('.csv')]

partial_recoveries = []

for file in filenames:
    partial = pd.read_csv(join(partial_output_folder, file))
    partial_recoveries.append(partial)

parameter_recovery = pd.concat(partial_recoveries).sort_values('index').reset_index(drop=True)
del partial_recoveries
parameter_recovery.to_csv(join(output_folder, 'parameter-recovery_{}.csv'.format(label)))
parameter_recovery.head()

,index,v_level,gamma_level,s_level,tau_level,v_gen,gamma_gen,s_gen,tau_gen,v_rec,...,gamma_rec,gamma_rec_hpd_lower,gamma_rec_hpd_upper,s_rec,s_rec_hpd_lower,s_rec_hpd_upper,tau_rec,tau_rec_hpd_lower,tau_rec_hpd_upper,converged
0,0,low,low,low,low,0.000028,-0.8,0.0047,0.215,0.000041,...,-0.750011,-0.845664,-0.657928,0.004512,0.004129,0.004960,0.252495,0.174107,0.328816,False
1,1,low,low,low,medium,0.000028,-0.8,0.0047,0.675,0.000036,...,-0.705180,-0.780706,-0.623233,0.004818,0.004141,0.005747,0.677458,0.543241,0.846280,False
2,2,low,low,low,high,0.000028,-0.8,0.0047,1.135,0.000054,...,-0.668112,-0.759103,-0.586810,0.005030,0.004491,0.005644,1.338666,0.842623,2.024470,False
3,3,low,low,medium,low,0.000028,-0.8,0.0075,0.215,0.000060,...,-0.853340,-0.950210,-0.731908,0.007088,0.006394,0.007782,0.245335,0.180926,0.316303,False
4,4,low,low,medium,medium,0.000028,-0.8,0.0075,0.675,0.000069,...,-0.627318,-0.723091,-0.533079,0.007746,0.006887,0.009003,0.734585,0.442170,1.269709,False
